In [1]:
# Import Librariesimport inspect
import time
from pprint import PrettyPrinter
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import seaborn as sns
from pymongo import MongoClient
import pytz
from statsmodels.tsa.ar_model import AutoReg
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA

# Prepare Data

## Connect

In [ ]:
# Connect to server
client = MongoClient(host=<"hostName">, port=<portNum>)

# Connect to database
db = client[<"databaseName">]

# Get collection
dar = db[<"collectionName">]

## Explore

In [ ]:
# Determine no. of sites in collection
sites = dar.distinct("metadata.site")     # dar ---> variable holding collection
sites

# Sample output
[11, 23]

In [ ]:
# Determing which collection has the most sensor readings using aggregate
result = dar.aggregate(
    [
        {"$group": {"_id": "$metadata.site", "count": {"$count":{}}}}
    ]
)
readings_per_site = list(result)
readings_per_site

# Sample output
[{'_id': 11, 'count': 173242}, {'_id': 23, 'count': 60020}]

## Import

In [ ]:
# Wrangle function
# Extract PM2.5 readings from collection site with the most readings
# Localize time
# Remove outliers
# Resample data to provide PM2.5 readings for each hour
# impute missing values
# return series

def wrangle(collection):
    results = collection.find(
        {"metadata.site": 11, "metadata.measurement": "P2"},
        projection={"P2": 1, "timestamp": 1, "_id": 0},
    )

    # Read data into DataFrame
    df = pd.DataFrame(results).set_index("timestamp")

    # Localize timezone
    df.index = df.index.tz_localize("UTC").tz_convert("Africa/Dar_es_Salaam")

    # Remove outliers
    df = df[df["P2"] < 100]

    # Resample to 1hr window
    y = df["P2"].resample("1H").mean().fillna(method='ffill')

    return y
  
# Using wrangle()
y = wrangle(dar)
y.head()

# sample output
timestamp
2018-01-01 03:00:00+03:00    9.456327
2018-01-01 04:00:00+03:00    9.400833
2018-01-01 05:00:00+03:00    9.331458
2018-01-01 06:00:00+03:00    9.528776
2018-01-01 07:00:00+03:00    8.861250
Freq: H, Name: P2, dtype: float64

## Explore Cont. 

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y.plot(xlabel="Date", ylabel="PM2.5 Level", title="Dar es Salaam PM2.5 Levels", ax=ax);

# sample output: see Figure 2.1 Time Series Plot

Insights:

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
y.rolling(168).mean().plot(ax=ax, xlabel="Date", ylabel="PM2.5 Level", title="Dar es Salaam PM2.5 Levels, 7-Day Rolling Average");
# --> 168 == number of hours in a week

# sample output: see Figure 2.2 Rolling Average 

Insights: 

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y, ax=ax)
plt.xlabel(<"xLabelvalue">)
plt.ylabel(<"yLabelvalue">)
plt.title(<"yourTitle">);

# sample output: see Figure 2.3 ACF Plot

Insights:

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))
plot_pacf(y, ax=ax)
plt.xlabel("Lag [hours]")
plt.ylabel("Correlation Coefficient")
plt.title("Dar es Salaam PM2.5 Readings, PACF");

# sample output: see Figure 2.4 PACF Plot

Insights:

## Split

In [ ]:
# percentage ---> 90% (0.9), 80% (0.8) ...
cutoff_test = int(len(y) * 0.9)
y_train = y.iloc[:cutoff_test]
y_test = y.iloc[cutoff_test:]
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

# sample output
y_train shape: (1944,)
y_test shape: (216,)

# Build Model

## Baseline

In [ ]:
y_train_mean = y_train.mean()
y_pred_baseline = [y_train_mean] * len(y_train)
mae_baseline = mean_absolute_error(y_train, y_pred_baseline)

print("Mean P2 Reading:", y_train_mean)
print("Baseline MAE:", mae_baseline)

# sample output
Mean P2 Reading: 8.57142319061077
Baseline MAE: 4.053101181299159

## Iterate

In [ ]:
# Use AR model to predict PM2.5 readings
# Hyperparameter --> p

# Create range to test different lags
p_params = range(1, 31)

# Create empty list to hold mean absolute error scores
maes = []

# Iterate through all values of p in `p_params`
for p in p_params:
    # Build model
    model = AutoReg(y_train, lags=p).fit()

    # Make predictions on training data, dropping null values caused by lag
    y_pred = model.predict().dropna()

    # Calculate mean absolute error for training data vs predictions
    mae = mean_absolute_error(y_train.iloc[p:], y_pred)

    # Append `mae` to list `maes`
    maes.append(mae)

# Put list `maes` into Series with index `p_params`
mae_series = pd.Series(maes, name="mae", index=p_params)

# Inspect head of Series
mae_series.head()

# sample output
1    1.059376
2    1.045182
3    1.032489
4    1.032147
5    1.031022
Name: mae, dtype: float64

In [ ]:
# Getting the mae_series to determine the best p that provides best performance
mae_series

# sample output
1     1.059376
2     1.045182
3     1.032489
4     1.032147
5     1.031022
6     1.026948
7     1.023510
8     1.022944
9     1.022321
10    1.022058
11    1.018001
12    1.018568
13    1.019006
14    1.018080
15    1.018818
16    1.021036
17    1.021398
18    1.021820
19    1.020472
20    1.019727
21    1.017981
22    1.017824
23    1.014569
24    1.013427
25    1.010657
26    1.010285
27    1.010619
28    1.010287
29    1.012046
30    1.016535
Name: mae, dtype: float64

best_p = 28
best_model = AutoReg(y_train, lags=best_p).fit()

In [ ]:
# Build and train the model
best_model = AutoReg(y_train, lags=best_p).fit()

In [ ]:
# calculate training residuals for best_model
y_train_resid = best_model.resid
y_train_resid.name = "residuals"
y_train_resid.head()

# sample output
timestamp
2018-01-02 07:00:00+03:00    1.674228
2018-01-02 08:00:00+03:00   -0.372994
2018-01-02 09:00:00+03:00   -0.535203
2018-01-02 10:00:00+03:00   -2.210960
2018-01-02 11:00:00+03:00    0.145636
Freq: H, Name: residuals, dtype: float64

In [ ]:
# Histogram of residuals
y_train_resid.hist()
plt.xlabel="Residuals"
plt.ylabel="Frequency"
plt.title="Best Model, Training Residuals";

# sample output: see Figure 2.5 Residuals Histogram 

Insights:

In [ ]:
# ACF Plot of residuals 
fig, ax = plt.subplots(figsize=(15, 6))
plot_acf(y_train_resid, ax=ax)
ax.set_xlabel("Lag [hours]")
ax.set_ylabel("Correlation Coefficient")
ax.set_title("Dar es Salaam, Training Residuals ACF");

# sample output: see Figure 2.6 Residuals ACF Plot 

Insights:

## Evaluate

In [ ]:
# walk-forward validation for model for test data --> y_test
# predictions stored in series: y_pred_wfv
y_pred_wfv = pd.Series()
history = y_train.copy()
for i in range(len(y_test)):
    model = AutoReg(history, lags=best_p).fit()
    next_pred = model.forecast()      # next value after end of history
    y_pred_wfv = y_pred_wfv.append(next_pred)
    history = history.append(y_test[next_pred.index])
    
y_pred_wfv.name = "prediction"
y_pred_wfv.index.name = "timestamp"
y_pred_wfv.head()

# sample output
timestamp
2018-03-23 03:00:00+03:00    10.383307
2018-03-23 04:00:00+03:00     8.268941
2018-03-23 05:00:00+03:00    15.172779
2018-03-23 06:00:00+03:00    33.480666
2018-03-23 07:00:00+03:00    39.576329
Freq: H, Name: prediction, dtype: float64

# Communicate Results

In [ ]:
# Put test and walk-forward validation values
# in a dataframe and plot df
df_pred_test = pd.DataFrame(
    {"y_test": y_test, "y_pred_wfv": y_pred_wfv}
)
fig = px.line(df_pred_test, labels={"value": "PM2.5"})
fig.update_layout(
    title="Dar es Salaam, WFV Predictions",
    xaxis_title="Date",
    yaxis_title="PM2.5 Level",
)

fig.show()

# sample output: see Figure 2.7 WFV Line Graph